# 1. Learning Deep Features via Congenerous Cosine Loss for Person Recognition  

## 1.1 Abstract

人物识别目的是在复杂的场景中识别跨越时间和空间的相同身份。在本文中提出了一种新方法，通过训练一个网络来获得鲁棒且较好的特征来解决这个问题。我们直接比较和优化了两个特征之间的余弦距离 - 扩大类差异并减少了类内的差异。我们通过最小化样本与它们的质心之间的余弦距离来提出一种连续的余弦损失。这样的设计降低了复杂度，并且可以通过使用标准化输入的 softmax 来实现。我们的方法也不同于以前的人物识别工作，我们不对测试子集进行额外的训练。通过测量测试集中几个身体区域的相似性来确定人的身份。实验结果表明，所提出的方法比之前 state-of-the-arts 的方法有着更高的准确率。  

## 1.2 Introduction  

随着智能手机和数码相机需求的不断增长，今天的人们会拍摄更多照片来记录日常生活和故事。这种趋势迫切需要智能工具从个人数据，社交媒体或互联网上获取数以千计的图像中跨越不同的时间和空间来识别同一个人。以前的工作已经证明，由于许多因素，诸如非正面，不同的光照，外观的可变性，身份纹理等，在这种不受约束的环境中人物识别仍然是一个具有挑战性的问题。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/1.jpg?raw=true)  

最近提出的 PIPA 数据集包含数千幅具有复杂场景和相似外观的图像。图像的亮度，大小和上下文变化很大，许多实例都有部分甚至没有面部。图 1 显示了来自训练和测试集的一些样本。以前的工作通过多线索，多层次的方式来识别同一个人，其中训练集仅用于提取特征并且后续分类器（SVM或神经网络）在 test 0 中被训练。识别系统在 test 1 组中进行评估。我们认为这样的做法在实际应用中是不可行的，因为第二次 test 0 的训练是附加的并且如果添加新样本需要重新训练。相反，我们的目标是提供一组鲁棒的，广义的特征表示，它们直接在训练集上训练，并通过测量测试集之间的特征相似度来识别人物。没有必要在测试 test 0 上进行训练，即使有新的数据进入，系统仍然运行良好。  

如图 1 底部所示，green box 中的女人与 light green box 中的女人戴着类似的围巾，她的脸部部分出现或者在某些情况下脸部不会出现。为了获得鲁棒的特征表示，我们针对不同的区域训练几个不同深度模型，并结合不同区域特征的相似度得分来预测他们的身份。我们的重要发现是，在训练期间，交叉熵损失不能保证类别内样本的相似性。它放大了不同类别之间的差异，并忽略了同一类别的特征相似性。为此，我们提出了一个连续的余弦损失，即 COCO，以扩大类间差异，并缩小类内差异。它通过测量样本与其集群质心之间的余弦距离来实现。此外，我们还将每个 region patch 与预定义的基本位置对齐，以进一步使特征空间中的类别内的样本更加接近。这种对齐策略也可能使网络不太容易过拟合。  

图 2 显示了我们提出的算法训练的 pipeline。图像中的每一个实例都用一个 ground truth head 标注，我们使用 RPN 框架分别训练一个脸部和人体检测器。然后使用人体姿势估计器来检测人的关键部位以定位上身区域。crop 四个区域后，我们进行仿射变换，将不同的 patch 从训练样本对齐到“基本”位置。分别在 PIPA 训练集上使用 COCO 损失训练四个深度模型来获取一系列鲁棒的特征。总而言之，这项工作的贡献如下：  

- 提出一个连续余弦损失来直接优化类别内部和跨类别样本间的余弦距离。它是以简单的 softmax 实现的，具有较低的复杂度。
- 设计了一个人体识别 pipeline，利用多个身体区域获得可区分的特征表达，而无需对测试集进行二次训练。
- 通过仿射变换将区域 patch 与基准位置对齐，以减少样本间的差异，使网络不易过拟合。  

## 1.3 Related Work  

基于图像的人物识别旨在识别日常生活照片中人物的身份，这些问题可能会因杂乱的背景而变得复杂。Anguelov 等首先通过提出一个马尔科夫随机场框架来解决这个问题，以结合所有情境提示来识别人的身份。最近，Zhang 等人为这项任务引入了一个大规模数据集。他们积累了由深层模型训练的 poselet 级别人物识别器的线索，以补偿姿势变化。Li 等在 LSTM 中嵌入场景和关系上下文，并将人的识别作为序列预测任务。  

人物重新识别是为了匹配摄像机中不同视角下的行人图像，并在视频中引发了许多重要的应用。现有的工作采用度量学习和中级特征学习来解决这个问题。 Yi 等用 Siamese 深度网络来学习图像对之间的相似性。人物识别与重新识别的主要区别在于数据流。前者是在不同的地点和时间识别同一个人。在大多数情况下，在不同场合中人物的外观会有很大的变化。后者是在连续的视频中检测人，这意味着外观和背景随着时间并没有很大的变化。  

深层神经网络近年来很大程度地推进了计算机视觉领域的发展，提高了图像分类，目标检测和目标跟踪等任务的性能。深度学习成功背后的精髓在于其在高维空间中的非线性复杂性的优越表现力和大规模数据集，深层网络可以在很大程度上学习复杂的模式和代表性特征。  

## 1.4 Algorithm  

所提出的 COCO 损失是针对每个身体区域进行训练以获得鲁棒的特征。  

### 1.4.1 Region detection  

利用四个区域 $r \in {1,...,4}$ 即脸部，头部，全身和上身来训练特征。先来介绍这些区域的检测。

Face: 先用 Faster RCNN 中的 RPN 预训练一个人脸检测器。数据为来自互联网的大约为300,000 张图像。网络结构是浅层 ResNet 模型，移除了 res_3b 之后的层并添加两个损失层（分类和回归）。然后，我们对 PIPA 训练集中的人脸采用 COCO 损失对模型进行微调。人脸检测器识别脸部的 m 个关键点（眼睛，眉毛，嘴巴等），并通过平移，旋转和缩放将检测到的脸部 patch 进行标准化。设 $p,q \in R^{m*2}$ 分别表示人脸模型检测的 m 个关键点和对齐结果。我们将 $P,Q$ 定义为两个仿射空间，然后仿射变换 $A:P \mapsto Q$ 定义为：  

$p \mapsto q = Ap + b$

其中 $A \in R^{m*m}$ 是 P 中的线性变换矩阵，$b \in R^{m*2}$是 Q 中的偏差。这样的对齐方案可以确保类别内和类别之间的样本没有很大的差异：如果网络是在没有对齐的情况下学习的，它必须区分更多的模式，例如人员之间不同的旋转，更容易出现过拟合；如果数据进行了对其，则可以更侧重于区分不同身份的功能，而不用管旋转，视角等。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2.jpg?raw=true) 

Head, whole body and upper body：头部区域是作为每个人的 ground truth 给出的。为了检测整个身体，我们还在 RPN 框架中预先训练了一个检测器。 该模型在大型人类数据集上进行了训练，我们使用 4638717 副图像中的 87021 个人进行训练。网络结构是一个 inception 模型，最终的 pooling 被完全连接层所取代。为了确定上身区域，我们进行了人体姿态估计以识别身体的关键点，并由此确定上半身的位置。头部，全身和上半身的检测器基于 PIPA 训练集用 COCO loss 进行了微调，并做了面部部区域所述的类似的 patch 对准过程。图 2（c）显示了四个区域的对齐 patch。  

### 1.4.2  Congenerous cosine loss for training  

设计 COCO 损失是为了直接比较和优化两个特征之间的余弦距离。令 $f^{i,f} \in R^{D*1}$ 表示第 i 个样本在区域 r 的特征向量，其中 D 是特征维数。为简洁起见，我们去掉上标 r，因为每个区域模型都经历了相同的 COCO 训练。我们首先定义一个小批次 $B$ 中两个特征的余弦相似性，如下所示：  

$C(f^{(i)},f^{(j)}) = \frac {f^{(i)}*f^{(j)}} {\parallel f^{(i)} \parallel \parallel f^{(j)} \parallel} $

余弦相似度衡量两个样本在特征空间中的距离。理想的损失是增加类内样本的相似度，并扩大类间质心的距离。令 $l_i,l_j \in {1,...,K}$ 为样本 $i,j$ 的标签，其中 K 为类别总数，我们最大化下面的损失：  

$ L^{naive} = \sum_{i,j \in B} \frac {\delta (l_i,l_j) C(f^{(i)},f^{(j)})}{1 - \delta (l_i,l_j) C(f^{(i)},f^{(j)}) + \epsilon}$

其中$\delta(.,.)$ 是一个指示函数。这样的设计在理论上是合理的，但仍然存在计算效率低下的问题。此外，如果我们直接计算来自小批次的两个任意样本的损失，网络会遭受不稳定的参数更新，并且很难收敛。  

受中心损失的启发，我们将类别 k 的质心定义为小样本 B 上特征的平均值：  

$c_k = \frac {\sum_{i \in B} \delta (l_i,k) f^{(i)}}{\sum_{i \in B} \delta (l_i,k) + \epsilon} \in R^{D*1}$

结合上面两个公式，我们有以下样本 i 的输出需要去最大化：  

$p_{l_i}^{(i)} = \frac {exp C(f^{(i)},c_{l_i})}{\sum_{k \neq l_i} exp C(f^{(i)},c_k)} \in R$

公式通过类别质心来测量一个样本与其他样本的距离，而不是上面一样直接进行两两比较。确保样本 i 与其自己的类别 $l_i$ 足够接近。指数运算是将余弦相似度转换为归一化概率输出，范围从 0 到 1。  

为此，我们提出了cogenerous cosine（COCO）损失，这是为了增加类内的相似性，并以合作的方式扩大跨类别的变化：  

$ L^{COCO} = \sum_{i \in B} L^{(i)}  = -\sum_{k,i} t_k^{(i)} log p_k^{(i)} = - \sum_{i \in B} log p_{l_i}^{(i)}$

在实践中，可以通过 softmax 操作以一种简洁的方式实施 COCO loss：  

$p_{k}^{(i)} = \frac {exp (\hat{c}_k^T \cdot \hat{f}^{(i)})}{\sum_m exp (\hat{c}_m^T \cdot \hat{f}^{(i)})} = softmax(z_k^{(i)})$

其中 $z_k^{(i)} = \hat{c}_k^T \cdot \hat{f}^{(i)}$ 是softmax的输入。 $\hat{c}_k$ 可以看作是分类层中的权重，其中偏差项为零。  

请注意，这些特征和集群质心都是端对端训练的。这些特征是从预训练模型初始化的，并且 $c_k$ 的初始值是通过公式计算的。  

### 1.4.3  Relationship of COCO with counterparts  

COCO 损失被表述为特征空间中的度量学习方法，其使用余弦距离中的集群质心作为度量来扩大类间变化以及缩小类内差异。它可以通过几个限制条件下的softmax 操作来实现。图 3 显示了不同损失方案下的特征集群的可视化。对于 softmax 损失，它只强制跨类别的样本很远，而忽略一个类中的相似性。在COCO 损失中，我们用 softmax 之前的分类层来替换权重，并且有明确定义和可学习的集群质心。中心损失与我们的某些方面相似，但是，它需要一个外部存储器来存储类的中心，因此计算量是我们的两倍。Liu 提出了一个广义的 large-margin 损失，它也通过强化类内紧致性和类间可分性来学习判别性特征。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/3.jpg?raw=true) 

### 1.4.4   Inference  

在测试阶段，我们测量两个测试分片之间的特征的相似性，以基于 test_0 中的标签来识别 test_1 中的每个实例的身份。test_1 和 test_0 中的两个分片 i 和 j 之间的相似性由 $s_{i,j}^(r) = C(f^{(i,r)},f^{(j,r)})$，其中 r 表示特定的区域模型。关键问题是如何合并来自不同区域的相似度分数。 我们首先对初步结果$s^(r)$进行归一化处理，以使不同区域的分数具有可比性：  

$\hat{s}_{i,j}^{(r)} = (1 + exp [-(\beta_0 + \beta_1 s_{i,j}^{(r)})])^{-1}$

其中 $\beta_0, \beta_1$ 是逻辑回归的参数。最终得分 $S_{i,j}$ 是每个区域的归一化得分 $s_{ij}^{(r)}$ 的加权平均值。test_1 中的 patch i的身份由与参考集中的最大分数相对应的标签决定：$l_i = arg max_{j*} S_{i,j}$。 这样的方案保证了当新的训练数据被添加到 test_0 中时，不需要在测试集上训练第二个模型或SVM，这与以前的工作非常不同。

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/4.jpg?raw=true) 

# 2. Person Recognition in Social Media Photos  

## 2.1 Abstract  

现今人们通过社交媒体分享他们个人生活的大部分内容。通过简化相册管理，能够自动识别个人照片中的人物可能会大大提高用户的便利性。然而，对于人类识别任务来说，计算机视觉的传统焦点是人脸识别和行人重新识别。社交媒体照片中的人物识别为计算机视觉带来了新的挑战，包括复杂的主题（靠后的视角，不寻常的姿势）和外观上的巨大变化。为了解决这个问题，我们利用来自多个图像区域（头部，身体等）的卷积特征构建了一个简单的人物识别框架。我们提出新的识别方案，重点关注训练和测试样本之间的时间和外观的差距。我们根据时间和视角的一般性对不同特征的重要性进行深入分析。在此过程中，验证了我们的方法实现了 PIPA 基准测试集上最先进结果，这是迄今为止最大的基于社交媒体的人物识别基准，具有多种姿势，视角，社交群组和事件。  

与本文的会议版本相比，本文增加了(1)人脸识别器（DeepID2+）的分析；(2)结合会议版本方法 naeil 和 DeepID2 + 的新方法 naeil2，实现了 state of the art 的效果；(3)讨论自会议版本以来的相关工作，...。  

## 2.2 Introduction  

随着社交媒体的出现和图像捕捉模式从数码相机向智能手机和生活记录设备的转变，用户现在在线共享大量个人照片。通过简化相册管理，能够自动识别个人照片中的人物可能会大大提高用户的便利性。识别自然环境中的人物是一个有趣的挑战，人们可能将注意力集中在看不见人脸的活动上，或者可以改变服装或发型。这些挑战在很大程度上是新的 - 用于人类识别的传统计算机视觉研究的焦点是人脸识别（正面，完全可见的人脸）或行人重新识别（没有衣服变化，站立姿势）。  

表面上看，识别自然环境中的人脸是这个任务中的重要因素。然而，当人们参与一项活动（即不摆姿势）时，他们的面部只会变得部分可见（非正面，遮挡）或完全不可见（背面）。因此，需要额外的信息来可靠地识别人员。我们探索其他线索，包括（1）包含形状和外观信息的人的身体;（2）性别和年龄等人的属性;（3）场景上下文。图 1 给出了几个需要更多线索才能正确识别的例子。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/1.jpg?raw=true) 

本文对社交媒体类型照片中的人物识别任务进行了深入分析：给每个人一些标注的训练图像，这个人在测试图像中是谁？ 本文的主要贡献总结如下：  

- 提出具有挑战性的 PIPA 数据集人物识别方案。
- 详细分析不同线索的信息，特别是人脸识别模块DeepID2 +。
- 确认我们的期刊版本最终模型 naeil2 在 PIPA 上实现了最好的性能。
- 根据外观和视点变化分析线索的贡献。
- 在 openworld 识别设置下讨论我们的方法的性能。
- 开放代码和数据：https://goo.gl/DKuhlY 

## 2.2.1 Related work  

#### Data type  

以前关于人物识别的大部分工作都集中在人脸上。室外标记人脸（LFW）一直是实验室环境以外的许多关于人脸识别和验证工作的极好测试平台。近年来，测试基准的准确率已经接近饱和，归因于在大规模人脸数据库训练的深层特征传统的方法比基于手工特征和度量学习方法的复杂分类器上有很大的提高。然而，LFW 对于社交媒体照片并不具有代表性：数据主要由未被遮挡的正面（面部检测）组成，并且数据集集中在公众人物上。事实上，最近的基准数据已经引入了更多复制类型的数据。IARPA Janus Benchmark A (IJB-A) 包含了人物的侧脸等，但仍限于公众人物。  

不仅人脸，身体也被作为人物识别的线索进行了研究。例如，行人重新识别（re-id）解决了在不同摄像机视图中匹配行人检测的问题。标准 benchmarks 包括VIPeR，CAVIAR，CUHK 和 Caltech Roadside Pedestrians等，以前主要研究设计良好的手工特征，现在更多地关注于开发有效的深度特征。然而，典型的 re-id benchmarks 不能完全覆盖社交媒体图片的三个方面：（1）人物主要以站立姿势出现；（2）分辨率低；（3）匹配仅在短时间跨度内评估。  

日常自然环境中的人体识别首先由“Gallagher收集的人物数据集”涵盖。 然而，与典型的社交媒体帐户的大小相比，该数据集很小（〜600个图像，32个身份），并且只有正面被标注。 MegaFace 也许是社交媒体照片上最大的已知开源脸谱数据库。 但是，MegaFace 不包含任何背影数据（由面部检测器修剪），并且由于数据处理步骤而不保留每个帐户统计信息（例如，每个帐户的照片数量）。我们在 PIPA 数据集上构建论文，也从 Flickr 爬取，规模相当大（约40k图像，约2k个身份），具有不同的外观、所有视角和遮挡水平的人体。头部用带有身份标签的边界框进行标注释。我们在下一章中更详细地描述了 PIPA。  

#### Recognition tasks  

存在多个与人物识别相关的任务，主要在训练和测试数据量方面存在差异。面部识别和监控重新识别通常通过验证完成：给定一个参考图像（图库）和一个测试图像，它们是否是同一个人？。在本文中，我们考虑两个识别任务。(1) 封闭的识别任务：给定一个测试图像，这个人是训练集中的哪个人？(2) 开放的识别任务：给出测试图像，这个人是否是训练集中的人，如果是，是谁？  

其他相关的任务是人脸聚类，寻找重要人物，或者将文本中的名字与图像中的人脸相关联。  

#### Prior work with the same data type and task  

自从引入 PIPA 数据集以来，多个作品提出了不同的方法来解决社交媒体照片中的人物识别问题。 Zhang 等人提出了姿态不变人识别（PIPER），通过结合三个要素来获得有希望的结果：DeepFace（在大型私人数据集上训练的人脸识别模块）；poselets (在 2K 图片中 19 个关键点上训练的姿势估计模块)，以及 19 个关键点的卷积特征。  

Oh 等人，即本文的会议版本提出了一个简单的模型 naeil 从多个固定图像区域提取 AlexNet 特征。这个模型不需要数据需求量大的 DeepFace 或费时的Poselet，同时实现了比 PIPER 略好的识别性能。  

会议版本之后又有很多工作。 Kumar 等人通过使用姿态估计来标准化身体姿势从而改善性能。Li 等人考虑利用人物共现统计。Liu 等人已经提出训练一个度量空间中人物的特征向量，而不是在一组固定的身份上训练一个分类器，从而使得该模型更适应于陌生样本。一些作品利用了相册元数据，允许模型对不同的照片进行推理。  

在这个期刊版本中，我们使用 naeil 和 DeepID2+ 构建了 naeil2，以实现关于 PIPA 已发表的工作成果。 我们根据时间和视角变化提供线索的进行了更多的分析。  

## 2.3  Dataset and experimental setup  

#### Dataset  

据我们所知，PIPA 数据集（“人像摄影集”）是第一个用于注释人们身份的数据集，即使图片只是人物的背影。注释者标记了即使对于人类也可能被认为很难的实例。PIPA 拥有 Flickr 中个人相册图像，共有 63188 个头部标注，共 2356 个身份。数据集被划分为 train，val，test 和剩余集，粗略比例为45：15：20：20。本文不使用剩余集合。  

#### Task  

在测试时，给一张测试图片和该图像中人物头部的 bounding box，任务是在给定的一组身份（图库集，200〜500个身份）中选择出该测试样本中人物的身份。  

在附录 B 中，我们评估测试实例可能是背景人员的方法（例如旁观者 - 没有给出训练图像）。然后系统还需要确定给定的实例是否在可见的身份(图库集)之中。  

#### Protocol  

我们遵循 PIPA 的协议进行数据训练和模型评估。训练集用于卷积特征训练，测试集包含测试身份的示例。对于每个身份，样本被分为 test0 和 test1。 为了评估，我们通过对其中一个分割进行训练并对另一个进行测试来执行双重交叉验证。验证集同样分为 val0 和 val1，用于探索不同的模型和调整超参数。  

#### Evaluation  

我们使用识别率（或准确率），即测试实例中正确的身份预测率。对于每个实验，我们平均从（训练，测试）对（val0，val1）和（val1，val0）获得的两个识别率 - 类似于测试。  

### 2.3.1 Splits  

我们考虑四种不同的方式来分割每个身份的训练和测试样本 ($val_{0/1} 和 test_{0/1}$)，目的是评估不同级别的泛化能力。第一个来自以前的工作，我们介绍三个新的工作。数据统计参见表 1，图 3 用于可视化。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/3.jpg?raw=true) 

#### Original split O  
 
原始分割的数据集对每个身份有很多相似的图片 - 例如连续拍摄的照片。因此原始分割很容易 - 即使原始 RGB 像素的最近邻分类也能工作。为了评估长期外观变化的泛化能力，我们在下面介绍三个新的分割。  

#### Album split A  

根据相册元数据为每个身份划分训练和测试样本。在尝试匹配每个身份的样本数量以及整个分组样本的总数时，每个分组都会获取相册。为了匹配样本数量，在分组之间共享几个相册。由于 Flickr 相册是用户定义的，并不总是严格聚集事件和场合，所以可能拆分并不完美。  

#### Time split T  

根据拍摄照片的时间分割样本。对于每个身份，样本都根据其“照片拍摄日期”元数据进行排序，然后根据最新与最旧的时间进行划分。没有时间元数据的实例根据均匀分布划分。这个分割评估识别器的时间泛化。然而，“照片拍摄日期”元数据非常嘈杂，也缺失数据。  

#### Day split D  

Day split 通过视觉检查来分割实例，以确保不同的分割之间存在外表的变化。我们定义了两个划分标准：（1）日期变化的确切证据，例如{变化的季节，地点，事件，共同发生的人} 和/或（2）{发型，化妆，头部或身体穿着}。我们放弃无法进行此类划分的身份。分割后，对于每个身份我们随机丢弃较大分组中的样本，直到大小匹配。 如果较小的分割 ≤ 4 个实例，则完全丢弃标识。 Day split 分组使实验能够评估整个外观和事件变化的泛化性能。  

### 2.4.2 Face detection  

PIPA 中的实例对头部进行了标注。我们另外计算了 PIPA 上的面部检测，其目的有三个：（1）比较头部与面部的身份信息量，（2）获得头部方向信息以供进一步分析和（3）在测试时模拟没有头部标注的场景。我们使用开源的 DPM 人脸检测器。  

给定一组检测到的人脸（高于特定检测分数阈值）和标注的头部，根据重叠（重叠的交集）进行匹配。对于匹配的头部，相应的人脸检测告诉我们 DPM 哪个部件被触发，从而允许我们推断头部方向（正面或侧面视图）。  

根据 DPM 组件，我们将 PIPA 中实例按照以下特点进行分割：（1）检测到和正面（FR，41.29％），（2）检测到和非正面（NFR，27.10％）和（3）未检测到面部（NFD ，31.60％）。我们将没有与标注头部匹配到的结果定义为没有检测到人脸。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/2.jpg?raw=true) 

## 3 Cues for recognition  

在本节中，我们研究了在社交媒体照片中识别人物的线索。我们首先概述我们的模型。然后，我们通过实验回答以下问题：固定的身体区域（无姿势估计）有多多少信息？ 场景上下文有多大帮助？ 头部还是脸部（头部减去头发和背景）更具更多的信息？ 通过使用扩展数据获得了多少信息？ 专业人脸识别器的效果如何？ 本节中的研究仅基于 val 数据集。  

### 3.1 Model overview  

在测试时，根据给定的头部标注框，我们估计图 4 中描绘的五个不同区域。每个区域被送到一个或多个神经网络以获得一组线索。这些线索拼接起来形成一个描述该实例的特征向量。在整篇论文中，我们写 + 来表示向量级联。线性 SVM 分类器通过这个特征向量进行训练（one versus the rest）。在我们的最终系统中，除了 DeepID2 + 以外，所有特征都使用经过 ImageNet 分类预训练的 AlexNet 的第七层（fc7）进行计算。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/4.jpg?raw=true) 

### 3.2 Image regions used  

我们根据头部标注框选择五个不同的图像区域（在测试时给出）。头部矩形 h 是真实头部标注框。全身矩形 b 定义为（3×头宽，6×头高），头部位于全身顶部中心。上半身矩形 u 是 b 的上半部分。场景区域是包含头部的整个图像。  

使用上节讨论的 DPM 人脸检测器获得人脸区域 f。对于没有匹配到人脸的的头部（例如背部视图和脸部遮挡），我们根据训练数据上的脸部位移统计从头部回归脸部区域。图 4 中显示出了五个相应的图像区域。  

请注意，这些区域彼此重叠，并且取决于人物的姿势。例如，对于躺着的人而言，b 可能包含比实际身体更多的背景。  

### 3.3 Fine-tuning and parameters  

除非另有说明，否则 AlexNet 都使用 PIPA 训练集（〜30k 实例，〜1.5k 身份）进行微调，裁剪五个不同的图像区域，并进行 30 万次小批量迭代（批量为50）。我们将这样得到的基本线索称为 f，h，u，b 或 s，这取决于裁剪区域。 在 val 数据集上，我们发现微调提供了比非微调 AlexNet 增加约 10％的准确率。我们使用 AlexNet 的第七层（fc7）（4096尺寸）提取特征。  

我们针对每个身份训练一个具有正则化参数 C = 1 的 one-versus-all SVM 分类器; 它在我们的初步实验中是一个不敏感参数。作为替代，我们也使用了最近邻分类器。然而，在 val 集上，SVMs 的表现一直最近邻分类器好 10 pp 左右。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/5.jpg?raw=true) 

### 3.4 How informative is each image region?  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/6.jpg?raw=true) 

表 2 显示了在验证集上每个区域单独的结果过区域特征结合起来的效果。头 h 和上身 u 是最强的个人表示。上半身比全身 b 更可靠，因为下半身通常被遮挡或从框架中切出，因此通常是干扰项。不出意外，场景 s 是最弱的个人表示，但它仍然是人物识别的有用信息（远高于偶然级别）。重要的是，尽管像素重叠，但我们看到所有线索都相互补充。总的来说，我们的特征组合策略是有效的。  

### 3.5 Scene (s)  

除了 fine-tuned 的 AlexNet，我们考虑了多种特征类型来编码场景信息。$S_{gist}$：使用 Gist 描述符（512维）。$S_{0 \ places}$：不考虑使用在 ImageNet 上预先训练的 AlexNet，而是考虑在“Places数据库”（约250万幅图像）的 205 个场景类别上预先训练 AlexNet（PlacesNet）。 $S_{places \ 205}$：我们也考虑使用每个场景类别（205个维度）的得分向量来代替 4096 维度的 PlacesNet 特征向量。 $S_0,S_3$：最后我们考虑以与身体或头部相同的方式使用 AlexNet（在PIPA 人物识别训练集上进行零或 300k 次微调迭代）。 $S_{3 \ places}$：进行了人物识别微调的 $S_{0 \ places}$ 。  

#### Results  

表 3 比较了 val 集的不同方案的结果。Gist 描述符 $S_{gist}$ 只比卷积特征效果略差（我们也尝试 了Gist 的4608维度版本，获得更差的结果）。使用 $S_{0 \ places}$ 的原始特征向量优于 $S_{places \ 205}$ 的类别分数。有趣的是，在这种情况下，位置分类的预训练比针对对象分类的预训练（$S_{0 \ places}$ 对 $S_0$）更好。经过微调 $S_3$ 达到与 $S_0$ 相似的表现。  

尝试不同组合的实验表明这些特征之间几乎没有互补性。由于 $S_0$ 和 $S_3$ 之间没有很大的差别，为了简单起见，我们在所有其他实验中使用 $S_3$ 作为场景线索。  

#### Conclusion  

尽管场景本身很弱，但可以获得远高于随机水平的结果。经过微调后，场景识别作为预训练替代任务不能提供明显的（ImageNet）物体识别增益。  

### 3.6  Head (h) or face (f)?  

大部分人脸识别工作都专注于脸部区域。在相册的背景下，我们的目标是量化头部与脸部区域的信息量。如上节所述，我们从 DPM 人脸检测器获得人脸区域 f。  

#### Results  

表 2 中 f 和 h 之间的差距大约为10％，突出了将头发和背景包含在脸上的重要性。  

#### Conclusion  

使用 h 比 f 更有效，但 f 仍有较好的表现。与其他身体线索一样，h 和 f 之间存在互补性，建议一起使用它们。  

### 3.7 Additional training data ($h_{cacd}, h_{casia}$)  

众所周知，深度学习架构可从其他数据中受益。PIPER 使用的 DeepFace 在超过$4 \cdot 10^3$人（私人 SFC 数据集）的 $4.4 \cdot 10^6$ 个人脸上进行训练。相比之下，我们的线索是通过 ImageNet 和 PIPA 超过 $1.4 \cdot 10^3$ 人的 $29 \cdot 10^3$ 个人脸上进行训练。为了测量大数据对训练的影响，我们考虑使用两个开源人脸识别数据集进行微调：CASIA-WebFace（CASIA）和“Cross-Age Reference Coding Dataset”（CACD）。  

CASIA 包含$10.5 \cdot 10^3$ 人的 $0.5 \cdot 10^6$ 个人脸图片（主要是演员和公众人物）。当对这些身份（使用头部区域 h）进行微调 AlexNet 时，我们得到 $h_{casia}$ 线索。  

CACD 包含$2 \cdot 10^3$ 不同年龄人的 $160 \cdot 10^3$ 个人脸图片。尽管图像总数量少于 CASIA，但 CACD 的每个身份（~2×）具有更多的样本数量。$h_{cacd}$ 线索是通过与 $h_{casia}$ 相同的程序构建的。  

#### Results  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/7.jpg?raw=true) 

结果见表 4 的上半部分。 $h + h_{cacd}$和 $h + h_{casia}$ 改善超过 h（分别为 1.0 和 2.2 pp）。额外的训练数据似乎有所帮助。然而，由于数据分布不匹配，$h_{cacd}$ 和 $h_{casia}$ 本身比 h 差约 5 pp  

#### Conclusion  

即使是来自不同类型的照片，额外的训练数据也可以提供帮助。  

### 3.8  Attributes ($h_{pipa11}, u_{peta5}$)  

虽然整体外观可能每天都在变化，但可以预期，长期和稳定的属性提供了识别手段。我们通过微调 AlexNet 特征来构建属性线索，不是针对人物识别任务，而是针对属性预测替代任务。我们考虑两个属性，一个在头部区域，另一个在上身区域。  

我们在 PIPA 训练和验证集（1409 + 366）中注明了身份，包含五个长期属性：年龄，性别，眼镜，头发颜色和头发长度。我们通过微调 AlexNet 特征来构建头部属性预测任务的 $h_{pipa11}$。  

为了微调属性线索 $h_{pipa11}$，我们考虑两种方法：针对所有属性训练单个网络作为具有 sigmoid cross entropy loss 的多标签分类问题，或者分别调整每个属性的一个网络并连接特征向量。val 组的结果表明后者（$h_{pipa11}$）比前者（$h_{pipa11m}$）表现更好。  

对于上半身属性特征，我们使用“PETA 行人属性数据集”。该数据集最初对$19 \ cdot 10^3$ 个行人全身图像标注 105 个属性。我们选择了五项长期属性作为研究对象：性别，年龄（年轻成人，成年人），黑发和短发。我们选择使用上身 u 而不是全身 b 来进行属性预测是因为 u 的噪声要小很多。我们使用属性预测任务对 PETA 图像上半部分的 AlexNet 特征进行训练，以获得 $u_{peta5}$ 特征。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/8.jpg?raw=true) 

#### Results  

见表 4 中的结果。PIPA（$h_{pipa11}$）和PETA（$u_{peta5}$）的结果相似（相对于 h 和 u，增益约为 1 pp），并显示互补性（相对于 h + u 约5 pp增益）。在考虑的属性中，性别对提高识别准确性贡献最大。  

#### Conclusion  

添加属性信息可提高性能。  

### 3.9  Conference version final model (naeil)  

本文会议版本的最终模型结合了 5 个普通区域线索（$P_s = P + s$），两个用额外数据训练的头部线索（$h_{cacd}, h_{casia}$）和 10 个属性线索（$h_{pipa11}, u_{peta5}$），共得到 17 个线索。我们将这个方法命名为naeil。  

#### Results  

结果见表 4。通过将所有简单的线索结合起来，在val set上达到最佳结果91.70％。  

#### Conclusion  

到目前为止考虑的线索是互补的，并且组合模型 naeil 是有效的。  

### 3.10 DeepID2+ face recognition module ($h_{deepid}$)  

近年来，人脸识别性能有了显著提高，具有更好的体系结构和更大的开源数据集。在本节中，我们研究了人脸识别对于人物识别有多少帮助。虽然 PIPER 使用的 DeepFace 可以与 PIPER 进行更直接的比较，但它不是公开的。因此我们选择了DeepID2 +人脸识别器。人脸识别技术仍在快速发展，人脸数据集越来越大，这一部分的分析将会低估当前和未来的人脸识别器。  

DeepID2 + 网络是一个孪生神经网络，以 25个不同的头部区域作为输入，联合验证识别损失作为损失函数。该训练基于由 CelebFaces +，WDRef 和LFW 组成的大型数据库 - 共计$1.3 \cdot 10^4$ 个人的 $2.9 \cdot 10^5$ 副人脸图片。在测试时，它集合了通过面部标志物检测获得的 25 个区域的预测，得到的输出是 1024 维头部特征，我们表示为 $h_{deepid}$。  

由于 DeepID2 + 始于面部标志检测，因此 DeepID2 + 特征不适用于例如遮挡或向后的头部。因此，在63188 个实例（83.42％）中只有 52709个 具有可用的DeepID2 + 特征，并且我们使用零向量作为其余特征。  

#### Results - Original split  

有关 $h_{deepid}$ 和相关组合的 val set 结果，请参见表 6。与普通头部特征 h 相比较，由于一些数据的人脸特征，$h_{deepid}$ 本身较弱（68.46％）。然而，当与 h 结合使用时，通过利用强 DeepID2 +人脸特征和视点鲁棒的 h 特征的信息，性能达到85.86％。  

由于特征维度不均匀（4096与1024），我们尝试在连接（$h⊕h_{deepid}$）之前对 h 和 $h_{deepid}$ 进行L2归一化。这进一步提高了3％（88.74％） - 优于 $h + h_{cacd} + h_{casia}$，这是头部区域以前的最佳模型（86.26％）。  

#### Results - Album, Time and Day splits  

表 6 还显示了 val 集上的相册，时间和日期分割结果。虽然一般头部线索 h 在 Day 分割中显著降低，但 $h_{deepid}$ 是一种可靠的线索，在所有四个分组（60〜70％）中识别水平大致相同。这并不奇怪，因为与头发，衣服和事件相比，脸部在很大程度上是不变的。  

在另一个子集中，h 和 $h_{deepid}$ 的互补性只有在它们在连接之前进行 L2 正则化时才能得到保证。 L2 正则化级联 $h⊕h_{deepid}$ 包含了所有分片上各个线索的表现。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/9.jpg?raw=true) 

#### Conclusion  

具有人脸特定架构/损失和大量训练数据的 DeepID2 + 为人物识别任务提供了非常有用的信息。但是，只能识别面部可见的实例，需要将其与定位结合起来以确保最佳性能。不出所料，专门的人脸识别器对训练和测试样本（相册，时间和日期分割）之间外观差距较大的情况帮助较大。更好的面部识别在未来将会进一步改善结果。  

### 3.11 Combining naeil with $h_{deepid}$ (naeil2)  

我们通过结合 naeil 和 $h_{deepid}$ 来构建期刊版本的最终模型，即 naeil2。从 3.10 可以看出，由于维数差异更大（4096×17 = 69632 与 1024），简单的级联很可能失败。我们 L2 正则化的 naeil 和 $h_{deepid}$，然后进行加权级联。  

$ naeil \bigoplus_{\lambda} h_{deepid} = \frac {naeil}{\parallel naeil \parallel_2} + \lambda \cdot \frac {h_{deepid}}{\parallel h_{deepid} \parallel_2}$

其中，$ \lambda > 0 $是一个参数，+ 表示一个级联。  

#### Optimisation of λ on val set  

$ \lambda $确定要给予 $h_{deepid}$ 多少的权重。正如我们在 3.10 中看到的，$h_{deepid}$ 的额外贡献量对于每个 split 都是不同的。在这一节中，我们在验证集的每个 split 上去计算$ \lambda $ 的最优值。由此产生的 naeil 和 $h_{deepid}$ 的组合是我们的最终方法 naeil2。 最优值在等距点 {0,0.05,0.1，...，3} 上搜索。  

请参阅图 6，了解具有 $ \lambda $ 变化值对性能的影响。对 Original, Album, Time, and Day splits 的最佳权重分别为 $ \lambda = [0.60 1.05 1.00 1.50] $,。 在外观变化很大的 splits 上面 $h_{deepid}$ 权重就很大。对于每个分割，我们将 naeil2 表示为 naeil 和 $h_{deepid}$ 基于最优权重的组合。  

请注意，对于所有分组中的 $ \lambda ≥1.5 $，性能曲线都相当稳定。在实践中，当受试者的预期外观变化量未知时，我们的建议是选择$ \lambda ≈1.5 $。最后，我们注意到，也可以对 naeil 中的 17 个线索完成搜索权重。  

#### Results  

请参阅表 6 了解 naeil 和 $h_{deepid}$ 的结果。 简单连接的 $Naeil + h_{deepid}$ 在 Day split 上表现比 $h_{deepid}$ 差（51.02％vs 60.46％）。但是，加权组合 naeil2 在所有四个分组中都实现了最佳性能。  

#### Conclusion  

当结合 naeil 和 hdeepid 时，加权组合是可取的，所得到的最终模型 naeil2 击败了所有四个 split 的先前考虑的模型。  

## 4  PIPA test set result and comparison  

在本节中，我们将测量我们最终模型在 PIPA 测试集和关键中间结果的性能，并与现有技术进行比较。总结见表 7。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2/10.jpg?raw=true) 

### 4.1  Baselines  

我们考虑两个基线来衡量任务的固有难度。 第一个基线是“机会水平”分类器，它没有看到图像内容，只是选择最常见的类。它为任何识别方法提供了下限，展示了图库集有多大。  

我们的第二个基线是原始 RGB 最近邻分类器 $h_{rgb}$。它使用原始尺寸缩小（40×40像素）和模糊的 RGB 头部裁剪作为特征。在测试时用欧几里得距离预测训练图像最近邻的身份。按照设计，$h_{rgb}$ 只能识别 test0/1 中头部几乎相同的人物。  

#### Results  

在表 7 中查看“机会水平”和 $h_{rgb}$ 的结果。虽然“机会水平”表现很低（在所有 split 中≤2％），但我们观察到$h_{rgb}$ 在原始 split （33.77％）表现的不可思议的好。这表明原始拆分在拆分中共享许多几乎相同的人员实例，而且任务非常简单。在较难分离的情况下，我们看到 $h_{rgb}$ 表现下降，在 Day 分组中仅达到 6.78％。因此，Day split 的识别就不那么简单了 - 仅仅利用像素值相似性就行不通。  

#### Conclusion  

尽管数据集集足够大，但通过在分组之间共享许多类似实例（原始分组），可以使任务变得容易。我们通过引入三个更具挑战性的分割（Album, Time, and Day）来解决这个问题，简单的 RGB 基线（$h_{rgb}$）不再有效。  

### 4.2 Methods based on head  

我们考虑了四个中间模型（$h，h + h_{casia} + h_{cacd}，h_{deepid}，h⊕h_{deepid}$）以及之前的作品 DeepFace。  

我们观察到与前面几节中在 val 集上相同的结果。在这里，我们将重点放在与 DeepFace 的比较上。即使没有专门的人脸模块，h 已经比 DeepFace 表现得更好（76.42％与46.66％，原始分割）。我们认为这有两个原因：（1）DeepFace 只将脸部区域作为输入，而没有有价值的头发和背景信息，（2）DeepFace 仅对脸部可以匹配的实例（52％）进行预测。请注意，由于未能估计姿势（PIPA失败17％），$h_{deepid}$ 也并不总是做出预测，但在所考虑的情况下表现要好于 DeepFace（68.06％比46.66％，原始分割）。  

### 4.3 Methods based on body  

我们考虑三种中间模型（$b，h + b，P = f + h + u + b$）和四种现有技术（GlobalModel，PIPER，Pose，COCO）。姿态和 COCO 方法出现在本文会议版发表后。结果见表 7。  

我们的身体线索 b 和 Zhang 等人的 GlobalModel 是独立实施的相同方法。不出所料，它们表现相似（分别为 69.63％ 和 67.60％，原始分割）。  

我们的 h + b 方法是 Zhang 等人的 PIPER 的最小系统匹配（83.36％比83.05％，原始分割）。h + b 的特征向量比 PIPER 小约 50 倍，并且不使用面部识别器或姿态估计器。  

事实上，PIPER 通过其中一个 poselets 捕捉头部区域。因此，h + b 从 PIPER 的“GlobalModel + Poselets”子集中提取线索，但表现更好（83.36％比78.79％，原始分割）  

#### Methods since the conference version  

Kumar 等人的姿势估计使用 PIPA 训练集上的额外关键点标注来生成姿态聚类，并单独训练每个姿态聚类（PSM，姿态特定模型）模型。通过执行一种姿势归一化形式，他们已经显着改善了结果：分别在原始和日期分裂上超过 2.27pp 和 10.19pp。  

刘等人的 COCO为人物识别任务提出了一种新的度量学习损失。度量学习通过在不需要重新训练的情况下识别不可见的身份，为基于分类器的方法提供了优势。他们还使用 FasterRCNN 探测器更准确地定位脸部和身体。与Pose 或 Naeil 相比，最终的表现在所有四个分组中都可以说是很好的。但是，应该注意的是，COCO 中的脸部，身体，上半身和全身特征基于 GoogleNetv3 和 ResNet50 - 这些数字与主要基于 AlexNet 的所有其他方法不完全可比。  

### 4.5 Methods based on full image  

我们考虑两个中间模型（$Ps = P + s，naeil = Ps + E$）和 Contextual，该方法出现在本文会议版后。  

我们的 naeil 表现比 PIPER 更好（86.78％比83.05％，原始分割），同时具有小 6 倍的特征向量，而不依赖于人脸识别器或姿态估计器。  

#### Methods since the conference version  

Li 等人利用人物共现统计来改善结果。它分别在原始和日期分割上执行 1.97 pp 和 12.81 pp，优于 naeil。但是，应该注意 Contextual 使用了人脸识别器 DeepID。我们发现专门的人脸识别器可以大大提高识别质量。  

### 4.5  Our final model naeil2  

naeil2 是 naeil 和 $h_{deepid}$ 的加权组合。观察到，通过在 naeil 上附加面部识别器模块，我们可以在 Album,Time, and Day splits 上实现最佳性能。尤其是在 Day 分组中，naeil2 比第二好的方法 COCO 提高了8.85 pp（表7）。在原始分割中，COCO 表现更好（2.36pp差距），但请注意，COCO 使用更高级的特征表示（GoogleNet 和 ResNet）。  

由于 naeil2 和 COCO 专注于正交技术，因此它们可以结合使用，以获得更好的性能。  

### 4.6 Computational cost  

我们在我们的方法中报告一些 pipelines 的计算时间。特征训练在单个 GPU 机器上需要 2-3 天。SVM 训练在原始分割（581个分类，6 443个样本）上对于 h（4096 dim）需要42秒，对于naeil需要 1118 秒。请注意，这对应于照片共享服务中的真实用户场景，其中用户已知约 500 个身份，并且每个身份的平均照片数量是 ~10。  

## 5 Analysis  

在本节中，我们将对最终结果的各个线索进行更深入的分析。特别是，当系统必须跨时间或头部视角时，我们测量单个线索（如脸部和场景）的贡献是如何变化的。我们将性能作为每个身份的训练样本数量的函数进行研究，并根据身份的可识别性来检查身份的分布。  

### 5.1 Contribution of individual cues  

我们通过将每个中间方法的精度除以 naeil2 的性能来衡量各个线索对最终系统 naeil2 的贡献。我们在四个分组中报告结果，以确定当训练样本和测试样本之间存在较大时间差时哪些线索贡献更多，反之亦然。  

#### Results  

有关四个分组的相对表现，请参见图 7。基于上下文（例如 b 和 s）的线索看到从 Original split 到 Day split 时下降很多，而侧重于面部 f 和头部 h 区域的线索倾向于更少下降。直观地说，这是由于 Day split 中的服装和事件发生了较大的变化。  

另一方面，$h_{deepid}$ Original split 到 Day split 的相对贡献增加很多，在 naeil2 几乎占了 90％。$h_{deepid}$ 提供了宝贵的面部不变特征，特别是当时差很大时。然而，在 Original split $h_{deepid}$ 只占 naeil2 的约 75％。应该增加头部方向健壮的 naeil 以获得最佳性能。  

#### Conclusion  

涉及上下文的线索在原始分割中更强; 面部表情，特别是 $h_{deepid}$，在 Day split 中表现强劲。结合两种类型的线索在所有考虑的时间/外观变化上产生最佳性能。  

### 5.2  Performance by viewpoint  

我们研究测试实例视角变化对提出方法的影响。依赖于脸部的线索不太可能对被遮挡的脸部具有鲁棒性，而身体或上下文线索对于视角变化将是强健的。 我们测量由 DPM 头部检测器定义的头部方向模型的性能：正面 FR，非正面 NFR 和未检测到面部 NFD。NFD子集是背视图和遮挡面实例的代表。  

#### Results  

图 8 显示了 Original split 和 Day split 的三个头部方向子集上方法的准确性。FR、NFR 、NFD 子集 上所有方法的性能依次下降。 然而，在Original split 中，即使对于 NFD 子集（大约80％的准确率），naeil2 仍然可靠地预测身份。在 Day split 中，naeil2 也在 NFD 子集上做出了努力（准确率达到20％）。Day split 下的 NFD 实例的识别构成了人员识别的剩余主要挑战。  

为了测量不同头部定向子集中各个线索的贡献，我们报告了相对于图 9 中的最终模型 naeil2 的相对表现。结果在 Original split 和 Day split 中报告。通常，当脸部不可见（例如 f 和 h）时，基于更多上下文（例如，b 和 s）的线索更健壮。请注意，NDET 中 $h_{deepid}$ 性能显着下降，而 naeil通常会在较难观察的情况下提高其相对性能。naeil2 包含了所有方向子集中各个线索的表现。  

#### Conclusion  

naeil 比 $h_{deepid}$ 更具视角稳健性，与时间稳健性分析形成鲜明对比。组合模型 naeil2 充分利用了这两个特点。其余人员识别挑战在于在 Day split 划分下没有发现 NFD 实例。也许可以利用图像或社交媒体元数据（例如相机统计，时间和 GPS 位置，社交媒体友谊图）。  

### 5.3 Generalisation across viewpoints  

在这里，我们研究我们模型的视角一般性。例如，我们挑战从只看到人脸的情况下从后面识别一个人。  

#### Results  

图 10 显示了方法的精确度，当它们仅在正面子集 FR（左图）或仅在无面部检测子集 NFD（右图）上进行训练时。在接受 FR 训练后，naeil2 难以泛化到NFD 子集（FR 与 NFD 性能约为原来约 95％〜40％;约8 5％〜约35％）。然而，表现仍远高于随机机会，表明学习到的身份表示在一定程度上是一般化的。 在这种情况下，naeil 功能比 $h_{deepid}$ 更强大，从 FR 到 NFD 的下降幅度更小。  

当训练期间没有面孔时（NFD子集上的训练），身份通常难于学习。即使在非泛化情况下，识别性能也很低：在 Original and Day 分别为〜60％和〜30％，这些都是在 NFD 上进行了训练和测试的。  

Conclusion  

naeil2 在各个角度都会略微泛化，主要归因于 naeil 特征。从背景或遮挡面孔（NFD）学习特定身份特征似乎相当困难。  

### 5.4 Viewpoint distribution does not matter for feature training  

我们检查了特征训练集中头部方向与头部特征 h 占比的影响。我们修改了仅由正面 FR 和非正面 NFR 构成的训练示例的数量，同时改变了它们的比例。  

可以假设每个人在 FR 和 NFR 的平衡混合物下达到该特征的最大视点鲁棒性; 同样，用 FR（NFR）子集训练的 h 在预测 FR（NFR）子集（分别）方面相对较强。  

#### Results  

图 11 显示了在 FR，NFR 和 NFD 子集上用各种 FR 到 NFR 比例训练的 h 的性能。与假设相反，改变特征训练中头部方向的分布对 Original and Day splits 中所有视点子集的表现影响 < 3％。  

#### Conclusion  

不需要额外的照顾来控制特征训练集中的头部取向的分布以改善头部特征 h。预计较大图像区域（例如 u 和 b）上的特征更不受视点分布的影响。  

### 5.5 Input resolution  

本节对输入分辨率的影响进行分析。我们的目标是确定在不同分辨率范围内方法是鲁棒的。  

#### Results  

图 12 显示了相对于输入分辨率（像素高度）的性能。最终的模型 naeil2 在低输入分辨率下具有很强的鲁棒性，即使对于在原始分割上头部具有 < 50 像素的情况，也能达到〜80％。在 day split，naeil2 在低分辨率示例（〜55％）上不太稳健。  

在组件方面，请注意，naeil 性能几乎不会影响分辨率级别。对于低分辨率输入，naeil往往比 $h_{deepid}$ 更强健，因为它基于身体和背景特征，并且不需要高分辨率的人脸。  

#### Conclusion  

对于低分辨率输入,应该用 naeil，而对于高分辨率输入,应该用 $h_{deepid}$。如果不确定，naeil2 是一个不错的选择 - 它包含了所有分辨率级别的性能。  

### 5.6 Number of training samples  

我们对两个问题感兴趣：（1）如果我们每个身份有更多的样本，人物识别是否会用当前的方法解决？（2）每个身份有多少例子足以收集关于一个人的大量信息？为了研究这些问题，我们测量了每个身份不同训练样本数的方法的性能。我们对每个数据点执行 10 次独立运行，每个标识具有固定数量的训练示例（子集在每次运行时均匀采样）。  

#### Results  

图 13 显示了不同训练样本个数下识别性能的趋势。naeil2 在 Original and Day splits 中每人 10 至 15 个训练样本饱和，分别达到 〜92％ 和 〜83％，每个身份 25 个样本。我们观察到每个身份的一个图片已经足以识别一个远高于机会水平的人（分别为原始和日期约67％和约35％）。  

#### Conclusion  

每人增加几次示例不会将性能提高到 100％。要完全解决问题，需要方法上的进步。另一方面，这些方法已经仅从每个人的单个样本（远高于机会水平）收集大量的身份信息。  

### 5.7 Distribution of per-identity accuracy  

最后，我们研究了多少比例的身份容易识别，有多少人不能被识别。我们通过根据身份识别精度计算身份的分布来研究这一点。  

#### Results  

图 14 显示了每个所考虑的方法按每个身份的准确性降序排列。在原始分割中，naeil2 为 581 个测试标识中的 185 个提供了 100％ 的准确性，而只有一个标识完全失败。另一方面，在 Day splits 中，naeil2 对199 个测试身份中有 11 个测试身份达到100％的准确率，12个身份完全失败。特别是，naeil2极大地提高了 naeil 对每个人的是被准确率，naeil 对 40 个身份完全失败。  

#### Conclusion  

在原始分割中，naeil2 已经在很多身份上表现良好。在 Day 拆分中，$h_{deepid}$ 特性大大提高了每个身份的表现，但 naeil2 仍然遗漏了一些身份。 将它留作未来工作重点。  

## 6 Conclusion  

我们分析了社交媒体照片中的人物识别问题，人们在这些照片中可能会出现遮挡的脸部，多姿多彩的姿势以及各种社交活动。我们研究了各种线索的效果，其中包括人脸识别器 DeepID2 +，以及它们的时间和头部视角的一般性。为了更好地进行分析，我们对 PIPA 做出了额外的分割，模拟了训练和测试样本之间不同的时间差异。  

我们得出了四个主要结论。（1）基于脸部和头部的线索在时间上很稳健。（2）基于上下文的线索在头部视点之间是健壮的。（3）最终模型 naeil2 是面部和上下文线索的组合，在时间和视点两方面都很强大，并且在具有挑战性的 Day split中，相对于最近的现有技术而言，实现了约 9 pp 的改进。（4）更好的细分网络体系结构和面孔识别器将在未来提高 naeil 和 naeil2 框架的性能。  

其余的挑战主要是时间差距大和遮挡的场景。一种可能的方向是利用诸如 GPS 和时间元数据，相机参数或社交媒体专辑/友谊图等非视觉提示。代码和数据可在https://goo.gl/DKuhlY上公开获取。


https://arxiv.org/abs/1702.06890